In [510]:
!pip install git+https://github.com/huggingface/transformers
!pip install wandb

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tji2ivgr
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-tji2ivgr
  Resolved https://github.com/huggingface/transformers to commit 799cea64ac1029d66e9e58f18bc6f47892270723
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
ERROR: Could not find a version that satisfies the requirement huggingface-hub<1.0,>=0.10.0 (from transformers) (from versions: 0.0.1, 0.0.2, 0.0.3rc1, 0.0.3rc2, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.1.0, 0.1.1, 0.1.2, 0.2.0, 0.2.1, 0.4.0)
ERROR: No matching distribution found for huggingface-hub<1.0,>=0.10.0
You should consider upgrading via the '/home/ecbm4040/envTF24/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ecbm4040/envTF24/bin/python3 -m 

In [511]:
!pip install torch transformers wandb -qqq

In [512]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            TextDataset, DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)
# from huggingface_hub.hf_api import HfAp
# import api.filter


import os
import numpy as np
import re
import pandas as pd
import tensorflow as tf
import torch
import pathlib
import random



import wandb
wandb.init(project="my-awesome-project")

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [513]:
#Tokenize:
file_path = "data/polite_hankgreen_50000_0.5_finalnum3541.txt"
wandb.run.name = file_path
"""
Polite Options:
polite_hankgreen_50000_0.5_finalnum3541.txt

Impolite Options:
impolite_hankgreen_50000_0.25_finalnum7746.txt



"""

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
block_size = tokenizer.model_max_length
train_dataset = TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=block_size, overwrite_cache=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#setup wanbi:
%env WANDB_PROJECT=tweet_analysis

wandb.run.name = file_path
wandb.run.save()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/ecbm4040/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_

env: WANDB_PROJECT=tweet_analysis


True

In [514]:
!nvidia-smi

Mon Dec 12 01:15:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    33W /  70W |  14068MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [515]:
#Train:
#Largely taken from: https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-

ALLOW_NEW_LINES = False     # seems to work better <--- from source
LEARNING_RATE = 1.372e-4
EPOCHS = 10
seed = random.randint(0,2**32-1)
training_args = TrainingArguments(
    report_to="wandb",
    output_dir="./model_files",
    overwrite_output_dir=True,
    do_train=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    prediction_loss_only=True,
    logging_steps=5,
    save_steps=0,
    seed=seed,
    learning_rate = LEARNING_RATE
    )

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset)
#LR schedule stuff?
train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader)
trainer.create_optimizer_and_scheduler(num_train_steps)
trainer.lr_scheduler = get_cosine_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps)

# p_start, p_end = 0.4, 0.9
# def progressify(f):
#     "Control progress bar when calling f"
#     def inner(*args, **kwargs):
#         if trainer.state.epoch is not None:
#             # we only have one epoch, EPOCHS is built into dataset
#             progress.value = p_start + trainer.state.epoch * (p_end - p_start)
#         return f(*args, **kwargs)
#     return inner

# trainer.training_step = progressify(trainer.training_step)



PyTorch: setting up devices
/home/ecbm4040/envTF24/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [516]:
trainer.train()
wandb.finish()


***** Running training *****
  Num examples = 432
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 432
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
5,4.240600
10,4.090700
15,4.129600
20,3.931100
25,3.984700
30,3.904400
35,3.965600
40,3.867600
45,3.866300
50,3.890700




Training completed. Do not forget to share your model on huggingface.co/models =)




train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▇▆▆▆▆▅▅▄▅▅▄▄▄▃▄▄▃▃▃▂▂▃▂▂▁▂▂▂▂▂▂▂▂▁▁▂▁▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,432


In [517]:
trainer.model.config.task_specific_params['text-generation'] = {
    'do_sample': True,
    'min_length': 15,
    'max_length': 100,
    'temperature': 100,
    'top_p': 0.95,
    'prefix': '<|endoftext|>'}

In [284]:
#Save Model:
trainer.save_model()

Saving model checkpoint to ./model_files
Configuration saved in ./model_files/config.json
Model weights saved in ./model_files/pytorch_model.bin
tokenizer config file saved in ./model_files/tokenizer_config.json
Special tokens file saved in ./model_files/special_tokens_map.json


In [518]:
a = trainer.state.log_history
print(a[0])

{'loss': 4.2406, 'learning_rate': 0.00013715465613551068, 'epoch': 0.01, 'step': 5}


In [286]:
## Import opposite politeness set:
#Tokenize:
file_path = "data/polite_hankgreen_50000_0.5_finalnum3541.txt"
#Polite:
# polite_hankgreen_50000_0.5_finalnum3541.txt
#Impolite:
# impolite_SenSanders_23400_0.5_finalnum9578.txt
# impolite_hankgreen_50000_0.25_finalnum7746.txt
with open(file_path) as f:
    lines = f.readlines()
lines = lines[0].split("<|endoftext|>")
lines[2]

"If you, as a child, received a rabbit foot keychain, what year were you born? I'm trying to figure out when we stopped giving children random animal parts."

# Predict:


In [527]:
# prepare input
predictions = []
start = "Dear John,"

"""
Notes:
Temperature: trade-off between variety and politeness clarity
Beam Search vs Top-k/Top-p. Beam search is a lot more coherent with a trade off for variety.

Options:

Naive Beam Search: Num_beams = 10, all else off
Top K with Nucleus Sampling: top_p = 0.95, top_k = 10-20, do_sample=True
Beam-search multinomial sampling : Num_beams = 10 + do_sample = True
Diverse beam-search decoding: Num_beams = 10 + num_beam_groups = 2





"""

start_with_bos = '<|endoftext|>' + start
encoded_prompt = trainer.tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)

# prediction
output_sequences = trainer.model.generate(
    input_ids=encoded_prompt,
    max_length=160, #originally 160
    min_length=10, #originally 10
    temperature= 0.75, #originally 1
    no_repeat_ngram_size=2,    
    
    
    #Edit stuff here down to change decoding 
# BEAM
#     num_beams=10, #on or off

# TOP    
#     do_sample=True, # for multinomial beam search and top sampling
#     top_p = 0.95, #0.95
#     top_k = 10, #10-20

# MULTINOMIAL
#     num_beams=10, #on or off
#     do_sample=True, # for multinomial beam search and top sampling

# DIVERSE 
#     num_beams=10, #on or off  
#     num_beam_groups = 2, # on or off, must be a multiple of num_beams

    
    
    num_return_sequences= 10, #must = num_beam for diverse beam-search
    
    )
generated_sequences = []

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = trainer.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    if not ALLOW_NEW_LINES:
        limit = text.find('\n')
        text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

for i, g in enumerate(generated_sequences):
    predictions.append([start, g])

for pair in predictions:
  print(pair[1])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/ecbm4040/envTF24/lib/python3.6/site-packages/transformers/generation_beam_search.py:197: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


Dear John, I just want you to know that I love you so much.
Dear John, I just want you to know that I love you.
Dear John, I just want you to know that I love you, John.
Dear John, I just want you to know that I love you so much. I am so sorry.
Dear John, I think it's time for you to get your ass out of the house.
Dear John, I think it's time for you to get your ass out of here.
Dear John, I just want you to know that I love you so much. I am so sorry about that.
Dear John, I think it's time for you to get out of the house.
Dear John, I think it's time for you to get your ass out there and do something good.
Dear John, I just want you to know that I love you so much. I am so sorry about this.


In [528]:
print("warning")

warning


In [529]:
username = "hankgreen" #twitter user
type_tweet = "polite" #polite or impolite
decoder = "diverse" #beam, top (top_k + top_p), multinomial (Num_beams + do_sample), diverse (Num_beams + num_beam_groups)

import pandas as pd
df = pd.DataFrame(columns = ["Target","Prompt","Tweets","Type"])
target_col = [username]*len(predictions) # target col
type_col = [type_tweet]*len(predictions) # type col
prompt_col = []
tweets_col = []

for pair in predictions:
    prompt_col.append(pair[0])
    tweets_col.append(pair[1])

df["Target"] = target_col
df["Prompt"] = prompt_col
df["Tweets"] = tweets_col
df["Type"] = type_col
df.reset_index()
# print(df)

df.to_csv('responses/{}_{}_{}.csv'.format(username,type_tweet,decoder), index=False)
#check writing:

df_test = pd.read_csv('responses/{}_{}_{}.csv'.format(username,type_tweet,decoder))
print(df_test)



      Target      Prompt                                             Tweets  \
0  hankgreen  Dear John,  Dear John, I just want you to know that I love...   
1  hankgreen  Dear John,  Dear John, I just want you to know that I love...   
2  hankgreen  Dear John,  Dear John, I just want you to know that I love...   
3  hankgreen  Dear John,  Dear John, I just want you to know that I love...   
4  hankgreen  Dear John,  Dear John, I think it's time for you to get yo...   
5  hankgreen  Dear John,  Dear John, I think it's time for you to get yo...   
6  hankgreen  Dear John,  Dear John, I just want you to know that I love...   
7  hankgreen  Dear John,  Dear John, I think it's time for you to get ou...   
8  hankgreen  Dear John,  Dear John, I think it's time for you to get yo...   
9  hankgreen  Dear John,  Dear John, I just want you to know that I love...   

     Type  
0  polite  
1  polite  
2  polite  
3  polite  
4  polite  
5  polite  
6  polite  
7  polite  
8  polite  
9  polite 

Empty DataFrame
Columns: [Unnamed: 0, Target, Tweets, Type]
Index: []


# New Section
